In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h1 style="background-color:LimeGreen; font-family:newtimeroman; font-size:200%; text-align:center; border-radius: 15px 50px;"> 1) Load Required Libraries </h1>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegressionCV
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

<h1 style="background-color:LimeGreen; font-family:newtimeroman; font-size:200%; text-align:center; border-radius: 15px 50px;"> 2) Read Data </h1>

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/test.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv")

In [ ]:
display(train.head())
display(test.head())
display(sub.head())

In [ ]:
print('size of train: ', train.shape)
print('size of test: ', test.shape)
print('size of submission: ', sub.shape)

In [ ]:
print(train.info())
print(test.info())

In [ ]:
train_original = train.copy()
test_original = test.copy()

<h1 style="background-color:LimeGreen; font-family:newtimeroman; font-size:200%; text-align:center; border-radius: 15px 50px;"> 3) EDA(Exploratory Data Analysis) </h1>

In [ ]:
train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [ ]:
train.columns.values

In [ ]:
features = list(train.columns)
list(enumerate(features))

<li style="font-family:'Goudy Old Style';font-weight: bold;font-size:30px;color: #c1531f  ">Describe Data</li>

In [ ]:
train.describe().style.background_gradient(cmap='bone_r')

<h1 style="background-color:orange; font-family:newtimeroman; font-size:180%; text-align:left; border-radius: 0px 0px;"> 3.1) Target Variable </h1>

In [ ]:
train['claim'].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

bars = ax.bar(train["claim"].value_counts().index,
              train["claim"].value_counts().values,
              color='darkorange',
              edgecolor="black",
              width=0.4)
ax.set_title("Claim (target) values distribution", fontsize=20, pad=15)
ax.set_ylabel("Amount of values", fontsize=14, labelpad=15)
ax.set_xlabel("Claim (target) value", fontsize=14, labelpad=10)
ax.set_xticks(train["claim"].value_counts().index)
ax.tick_params(axis="both", labelsize=12)

In [ ]:
plt.figure(figsize=(7,6))
sns.distplot(train['claim']);

<h1 style="background-color:orange; font-family:newtimeroman; font-size:180%; text-align:left; border-radius: 0px 0px;"> 3.2) Data Visualization </h1>

In [ ]:
fig, ax = plt.subplots(nrows = 2, ncols = 1, figsize=(9,8), sharex = True)

ax[0].scatter(train['f10'], train['f15'], ec = 'k', color = 'skyblue')
ax[0].set_ylabel('f15')
ax[0].set_title('Relation b/w f10 & f15')


ax[1].scatter(train['f3'], train['f4'], ec = 'k', color = 'skyblue')
ax[1].set_xlabel('f3')
ax[1].set_ylabel('f4')
ax[1].set_title('Relation b/w f3 & f4')

plt.tight_layout()
plt.show()

In [ ]:
df = pd.concat([train.drop(["claim"], axis=1), test], axis=0)
columns = df.columns.values

cols = 4
rows = len(columns) // cols + 1

fig, axs = plt.subplots(ncols=cols, nrows=rows, figsize=(16,130), sharex=False)

plt.subplots_adjust(hspace = 0.3)
i=0

for r in np.arange(0, rows, 1):
    for c in np.arange(0, cols, 1):
        if i >= len(columns):
            axs[r, c].set_visible(False)
        else:
            hist1 = axs[r, c].hist(train[columns[i]].values,
                                   range=(df[columns[i]].min(),
                                          df[columns[i]].max()),
                                   bins=40,
                                   color="deepskyblue",
                                   edgecolor="black",
                                   alpha=0.7,
                                   label="Train Dataset")
            hist2 = axs[r, c].hist(test[columns[i]].values,
                                   range=(df[columns[i]].min(),
                                          df[columns[i]].max()),
                                   bins=40,
                                   color="palevioletred",
                                   edgecolor="black",
                                   alpha=0.7,
                                   label="Test Dataset")
            axs[r, c].set_title(columns[i], fontsize=12, pad=5)
            axs[r, c].set_yticks(axs[r, c].get_yticks())
            axs[r, c].set_yticklabels([str(int(i/1000))+"k" for i in axs[r, c].get_yticks()])
            axs[r, c].tick_params(axis="y", labelsize=10)
            axs[r, c].tick_params(axis="x", labelsize=10)
            axs[r, c].grid(axis="y")
            axs[r, c].legend(fontsize=13)
                                  
        i+=1
plt.show();

<h1 style="background-color:LimeGreen; font-family:newtimeroman; font-size:200%; text-align:center; border-radius: 15px 50px;"> 4) Feature Engineering </h1>

<h1 style="background-color:orange; font-family:newtimeroman; font-size:180%; text-align:left; border-radius: 0px 0px;"> 4.1) Missing Values </h1>

In [ ]:
# print columns with highest number of Null values
train.isnull().sum().sort_values(ascending = False).to_frame().head(100).rename({0:'Counts'}, axis = 1).T.style.background_gradient('crest')

In [ ]:
fig, ax = plt.subplots(figsize=(28, 15))

bars = ax.bar(train.isnull().sum().index,
              train.isnull().sum().values,
              color="lightskyblue",
              edgecolor="black",
              width=0.7)
ax.set_title("Missing feature values distribution in the train dataset", fontsize=20, pad=15)
ax.set_ylabel("Missing values", fontsize=14, labelpad=15)
ax.set_xlabel("Feature", fontsize=14, labelpad=10)
plt.show()

In [ ]:
# print columns with highest number of Null values
test.isnull().sum().sort_values(ascending = False).to_frame().head(100).rename({0:'Counts'}, axis = 1).T.style.background_gradient('crest')

In [ ]:
fig, ax = plt.subplots(figsize=(25, 10))

bars = ax.bar(train.isna().sum().index,
              train.isna().sum().values,
              color="lightskyblue",
              edgecolor="black",
              width=0.7)
ax.set_title("Missing feature values distribution in the train dataset", fontsize=20, pad=15)
ax.set_ylabel("Missing values", fontsize=14, labelpad=15)
ax.set_xlabel("Feature", fontsize=14, labelpad=10)
plt.show()

<li style="font-family:'Goudy Old Style';font-weight: bold;font-size:30px;color: #c1531f  ">Impute Missing Values(SimpleImputer) with median as the strategy</li>

In [ ]:
imputer = SimpleImputer()

train_imputed = pd.DataFrame(imputer.fit_transform(train), columns=train.columns)
test_imputed = pd.DataFrame(imputer.fit_transform(test), columns=test.columns)

In [ ]:
# Checking the final dataset for missing values again.
pd.DataFrame(train, columns=train_original.columns).isnull().sum().sort_values(ascending = False).to_frame().reset_index().rename({0:'Count'}, axis =1).head(70).T.style.background_gradient('magma_r')

In [ ]:
# Checking the final dataset for missing values again.
test_imputed.isnull().sum().sort_values(ascending = False).to_frame().reset_index().rename({0:'Count'}, axis =1).head(70).T.style.background_gradient('magma_r')

<h1 style="background-color:orange; font-family:newtimeroman; font-size:180%; text-align:left; border-radius: 0px 0px;"> 4.2) Correlation b/n features </h1>

In [ ]:
train.corr()[['claim']].T.style.background_gradient('copper_r')

In [ ]:
corr_feat = train.corr()
plt.figure(figsize=(24,8))
corr_feat["claim"][:-1].plot(kind="bar", grid=True, color='#FF9505')
plt.title("Features correlation")

In [ ]:
# correlation matrix Using pandas
corr = train.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(1)

In [ ]:
plt.figure(figsize=(25,20))
ax = sns.heatmap(corr, vmin=-1, vmax=1, center=0,cmap=sns.diverging_palette(20, 200, n=200),square=True)

ax.set_xticklabels(ax.get_xticklabels(), horizontalalignment='right');

<h1 style="background-color:orange; font-family:newtimeroman; font-size:180%; text-align:left; border-radius: 0px 0px;"> 4.3) Outliers </h1>

In [ ]:
df_plot = ((train_imputed - train_imputed.min())/(train_imputed.max() - train_imputed.min()))

fig, ax = plt.subplots(4, 1, figsize = (25,25))
sns.boxplot(data = df_plot.iloc[:, 1:30], ax = ax[0])
sns.boxplot(data = df_plot.iloc[:, 30:60], ax = ax[1])
sns.boxplot(data = df_plot.iloc[:, 60:90], ax = ax[2])
sns.boxplot(data = df_plot.iloc[:, 90:120], ax = ax[3])

In [ ]:
def get_dist(df, col):
  plt.figure(figsize=(18,6))

  plt.subplot(1,3,1)
  plt.hist(df[col], bins=30)
  plt.title('Hist')

  plt.subplot(1,3,2)
  stats.probplot(df[col], dist='norm', plot=plt)
  plt.ylabel('quantiles')

  plt.subplot(1,3,3)
  sns.boxplot(y=df[col])
  plt.title('Boxplot')

  plt.show()

In [ ]:
get_dist(train, 'f9')

In [ ]:
get_dist(train, 'f45')

<li style="font-family:'Goudy Old Style';font-weight: bold;font-size:30px;color: #c1531f  ">Outlier Detection</li>

#### **Normal Distribution (Outliers)**

In [ ]:
def find_normal_outliers(df, col):
  upper_limit = train[col].mean() + 3*train[col].std()
  lower_limit = train[col].mean() - 3*train[col].std()

  return lower_limit, upper_limit

In [ ]:
find_normal_outliers(train, 'f1')

#### **Right Skewed (Outliers)**

In [ ]:
def find_skewed_outliers(df, col, distance):

  IQR = train[col].quantile(0.75) - train[col].quantile(0.25)

  upper_limit = train[col].quantile(0.75) + (IQR*distance)
  lower_limit = train[col].quantile(0.25) - (IQR*distance)


  return lower_limit, upper_limit

In [ ]:
find_skewed_outliers(train, 'f9', 1.5)

#### **Heavily Right Skewed (Outliers)**

In [ ]:
find_skewed_outliers(train, 'f110', 3)

<h1 style="background-color:LimeGreen; font-family:newtimeroman; font-size:200%; text-align:center; border-radius: 15px 50px;"> 5) Model Building and Evaluation </h1>

In [ ]:
X = train_imputed.drop('claim', axis=1)
y = train_imputed['claim']

In [ ]:
#### creating dataset split for prediction
X_train, X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42) # 80-20 split

#### Checking split 
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

In [ ]:
sc = StandardScaler()
sc.fit(X)
X_train_sc = pd.DataFrame(sc.transform(X), columns=X.columns)
test_sc = pd.DataFrame(sc.transform(test_imputed), columns=test.columns)

<li style="font-family:'Goudy Old Style';font-weight: bold;font-size:30px;color: #c1531f  ">Logistic Regressioncv</li>

- for more information refer: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html

In [ ]:
y_train_lr = train_original['claim']
X_train_lr = train_original.drop('claim', axis=1)

X_test_lr = test_original

In [ ]:
X_train_lr['n_missing'] = X_train_lr.isnull().sum(axis=1)
X_test_lr['n_missing'] = X_test_lr.isnull().sum(axis=1)

In [ ]:
columns_for_pred  = X_train_lr.columns
columns_for_pred

In [ ]:
IMPUTER = SimpleImputer()

X_train_lr =  IMPUTER.fit_transform(X_train_lr)
X_test_lr =  IMPUTER.fit_transform(X_test_lr)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train_lr)
X_train_lr = pd.DataFrame(scaler.transform(X_train_lr), columns=columns_for_pred)
X_test_lr = pd.DataFrame(scaler.transform(X_test_lr), columns=columns_for_pred)

In [ ]:
lr = LogisticRegressionCV(random_state=0)
lr.fit(X = X_train_lr, y = y_train_lr)

In [ ]:
lr.get_params()

In [ ]:
predict_lr = lr.predict(X_train_lr)
predict_proba = lr.predict_proba(X_train_lr)[:,1]

In [ ]:
m2_cm = confusion_matrix(y, predict_lr)
m2_acc_score = accuracy_score(y_train_lr, predict_lr)
print("Confusion Matrix")
print(m2_cm)
sns.heatmap(m2_cm, annot=True, cmap='CMRmap_r')
print("\n")
print("Accuracy of Logistic Regression:", round(m2_acc_score*100,2),'\n')
print(classification_report(y_train_lr, predict_lr))

In [ ]:
predict_lr_test = lr.predict_proba(X_test_lr)

<li style="font-family:'Goudy Old Style';font-weight: bold;font-size:30px;color: #c1531f  ">LGBM</li>

In [ ]:
LGBM =  LGBMClassifier(objective= 'binary', learning_rate = 0.05, max_depth = 3, n_estimators=500)
LGBM.fit(X_train, y_train)

In [ ]:
predict_LGB = LGBM.predict(X_test)

In [ ]:
m2_cm = confusion_matrix(y_test, predict_LGB)
m2_acc_score = accuracy_score(y_test, predict_LGB)
print("Confusion Matrix")
print(m2_cm)
sns.heatmap(m2_cm, annot=True,cmap='CMRmap_r')
print("\n")
print("Accuracy of LGBM:",round(m2_acc_score*100,2),'\n')
print(classification_report(y_test, predict_LGB))

<li style="font-family:'Goudy Old Style';font-weight: bold;font-size:30px;color: #c1531f  ">Submission</li>

In [ ]:
preds = LGBM.predict(test_imputed)

predictions = pd.DataFrame()
predictions["id"] = test_original["id"]
predictions["claim"] = predict_lr_test

predictions.to_csv('submission.csv', index=False, header=predictions.columns)
predictions.head()

In [ ]:
submission = pd.DataFrame({'id': pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv', usecols=['id'])['id'], 'claim': predict_lr_test[:,1]})
submission.to_csv('submission-lr.csv', index = 0)

In [ ]:
submission.head()